## Run cell below and click the link to open the website
Sometimes you have to run the cell twice. We are aware of the warning, it shouldn't matter, just click the link *below* the warning.

In [ ]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_leaflet as dl

from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

from functions.page_title import page_title
from functions.page_content import page_content

from sklearn.ensemble import RandomForestRegressor
import numpy as np
import joblib

JupyterDash.infer_jupyter_proxy_config()

#------------- website -------------#
app = JupyterDash(external_stylesheets=[dbc.themes.CERULEAN], suppress_callback_exceptions=True)
server = app.server
app.layout = dbc.Container(
    [
        dcc.Store(id="store"),
        html.H1("Air-BnB data New York", style={'margin-top': '10px'}),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dbc.Nav(
                   [
                        dbc.NavLink("Intro", href="/", active="exact"),
                        dbc.NavLink("1", href="/page-1", active="exact"),
                        dbc.NavLink("2", href="/page-2", active="exact"),
                        dbc.NavLink("3", href="/page-3", active="exact"),
                        dbc.NavLink("4", href="/page-4", active="exact"),
                        dbc.NavLink("5", href="/page-5", active="exact"),  
                        dbc.NavLink("6", href="/page-6", active="exact"),
                        dbc.NavLink("7", href="/page-7", active="exact"),
                        dbc.NavLink("8", href="/page-8", active="exact"),
                        dbc.NavLink("9", href="/page-9", active="exact"),
                    ],
                    pills=True,
                    ),
                    width='auto'
                ),
                html.Div(id="page-title", style={'width': 'auto', 'margin-right':'15px'}),                
            ],justify="between"),
        html.Div(id="page-content", className="p-4", style={
                'background-color': '#F5F5F5',
                'border-radius': '10px',
                'margin-top': '10px',
            }),
        html.Div([dcc.Location(id="url")]),
    ]
)

@app.callback(Output("page-title", "children"), [Input("url", "pathname")])
def render_page_title(pathname):
    return page_title(pathname)

@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    s1 = app.get_asset_url('img_slide5.png')
    s2 = app.get_asset_url('img_slide6.png')
    return page_content(pathname,s1,s2)

@app.callback(Output('prediction', 'children'), 
              [Input('slider1', 'value'),
               Input('slider2', 'value'),
               Input('slider3', 'value'),
               Input('slider4', 'value'),               
               Input("map-id", 'click_lat_lng')])
def prediction_update(room_type, min_night, avail, review, lat_lng):
    if lat_lng is None:
        lat = 40.730610
        lng = -73.935242
    else:
        [lat, lng] = lat_lng
    room = [0,0,0]
    if room_type == 0:
        room[0] = 1
    elif room_type == 5:
        room[1] = 1
    elif room_type == 10:
        room[2] = 1
        
    X = [lat, lng, min_night, review, 1, avail] + room
    reg = joblib.load('ML_models/model_forest_web.joblib')     
    price =  np.round(reg.predict([X,]).ravel())[0]
    return 'The suggested price is: {:0.0f}$'.format(price) 

@app.callback(Output("layer", "children"), [Input("map-id", "click_lat_lng")])
def map_click(lat_lng):
    if lat_lng is None:
        lat_lng = [40.730610, -73.935242]
    return [dl.Marker(position=lat_lng, children=dl.Tooltip("({:.3f}, {:.3f})".format(*lat_lng)))]


if __name__ == "__main__":
    app.run_server(debug=False, mode='external', port=8080)


In [ ]:
#app._terminate_server_for_port("localhost", 8080)